In [1]:
import os
import pandas as pd

from dateutil.relativedelta import relativedelta
import datetime
import calendar

import json

In [2]:
# !pip3 install tweepy

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
verbs = ['butuh', 'mau', 'ingin', 'pengen', 'kangen']
nouns = ['vacation', 'liburan', 'libur', 'staycation', 'kulineran', 'jalan jalan', 'traveling', 'cuti', 'wisata', 'piknik']

phrases = [f'{verb} {noun}' for verb in verbs for noun in nouns]
len(phrases)

50

In [5]:
def getMonthsBetween(date_from, date_until = datetime.date.today()):
    result = []

    today = date_until
    current = date_from

    while current <= today:
        result.append(current)
        current += relativedelta(months=1)
    return result

In [6]:
def getAllTweets(phrase):
    #get list of date
    dates = getMonthsBetween(datetime.date(2020, 1, 1), datetime.date.today())
    df_all = pd.DataFrame()
    for i in range(len(dates)):
        # define year months and next yearmonths
        #year_now = dates[i].year
        #month_now = dates[i].month
        date_nextmonth = dates[i] + relativedelta(months=1)
        
        # get months
        year_now = str(dates[i].year)
        month_now = str(dates[i].month).zfill(2)
        year_nextmonth = str(date_nextmonth.year)
        month_nextmonth = str(date_nextmonth.month).zfill(2)
        
        # define date_from and date_until
        date_from = "{}-{}-01".format(year_now, month_now)
        date_until = '{}-{}-01'.format(year_nextmonth, month_nextmonth)
        
        # cli_execute
        cli_command = 'snscrape --jsonl --since {1} twitter-search "{0} until:{2}"> text-query-tweets.json'.format(phrase, date_from, date_until)
        os.system(cli_command)
        
        # gathering all as one from temp
        df_temp = pd.read_json("text-query-tweets.json", lines = True)
        df_all = df_all.append(df_temp)
        
        #print
        #print(phrase, year_now + month_now, df_temp.shape[0])
        
    return df_all

In [7]:
# getAllTweets(phrases[0])

In [9]:
df_all = pd.DataFrame()
for phrase in phrases:
    df_all_temp = getAllTweets(phrase)
    df_all_temp['phrase'] = phrase
    df_all = df_all.append(df_all_temp)
    print(phrase, df_all_temp.shape[0])
df_all.to_pickle("df_all_tweets.pkl")

butuh vacation 315
butuh liburan 33733
butuh libur 10014
butuh staycation 1253
butuh kulineran 244
butuh jalan jalan 40500
butuh traveling 1084
butuh cuti 2918
butuh wisata 4132
butuh piknik 7588
mau vacation 2254
mau liburan 103520
mau libur 118147
mau staycation 14451
mau kulineran 5060
mau jalan jalan 618045
mau traveling 10505
mau cuti 36883
mau wisata 22103
mau piknik 11908
ingin vacation 155
ingin liburan 11565
ingin libur 8362
ingin staycation 1439
ingin kulineran 429
ingin jalan jalan 53923
ingin traveling 1745
ingin cuti 2653
ingin wisata 5876
ingin piknik 1176
pengen vacation 1072
pengen liburan 52675
pengen libur 42031
pengen staycation 7285
pengen kulineran 4571
pengen jalan jalan 135364
pengen traveling 6898
pengen cuti 8766
pengen wisata 4632
pengen piknik 5834
kangen vacation 374
kangen liburan 16387
kangen libur 10949
kangen staycation 1120
kangen kulineran 2508
kangen jalan jalan 59560
kangen traveling 2573
kangen cuti 1307
kangen wisata 1895
kangen piknik 3410


# cleansing and parsing

In [9]:
df_all = pd.read_pickle("df_all_tweets.pkl").drop_duplicates()

TypeError: unhashable type: 'dict'

In [8]:
df_all = pd.read_pickle("df_all_tweets.pkl").drop_duplicates()
df_all = df_all[df_all['id'].notna()]
df_all['id'] = df_all['id'].astype('int64')
print(df_all.shape)
# df_all.to_pickle("df_all_tweets.pkl")

TypeError: unhashable type: 'dict'

In [ ]:
# df_all = pd.read_pickle("df_all_tweets.pkl")
# df_all.shape

In [ ]:
def parseEverything(df):
    df = df[df.user != '0']
    def parseMentionedUser(string, subject):
        if str(string)!= 'nan':
            return [acc[subject] for acc in eval(str(string))]
        else:
            return None

    def tryEval(string, subject):
        try:
            return eval(string)[subject]
        except:
            print(string)
            return None
    list_columns = list(eval(df['user'][0]).keys())
    for column in list_columns:
        print(column)
        column_user = 'user_'+column
        df[column_user] = df['user'].apply(lambda x: tryEval(x, column))
    #df.head()
    df['mentioneduser_username'] = df['mentionedUsers'].apply(lambda x: parseMentionedUser(x, 'username'))
    df['mentioneduser_displayname'] = df['mentionedUsers'].apply(lambda x: parseMentionedUser(x, 'displayname'))
    df['mentioneduser_id'] = df['mentionedUsers'].apply(lambda x: parseMentionedUser(x, 'id'))
    df['mentioneduser_url'] = df['mentionedUsers'].apply(lambda x: parseMentionedUser(x, 'url'))
    df['count_mentionedusers'] = df['mentioneduser_username'].apply(lambda x: len(x) if x != None else 0)
    df = df.drop(columns = ['user', 'mentionedUsers'])
    return df

In [ ]:
df_all_parsed = parseEverything(df_all)
print(df_all_parsed.shape)
df_all_parsed.to_pickle("data_tweets_all_parsed.pkl")

In [ ]:
print('berhasil')

In [ ]:
# df_all[df_all['user'].apply(lambda x: eval(x)['username']).str.isnumeric()]